License
---
This file is part of CPAchecker,<br>
a tool for configurable software verification:<br>
https://cpachecker.sosy-lab.org<br>
<br>
SPDX-FileCopyrightText: 2020 Dirk Beyer <https://www.sosy-lab.org><br>
<br>
SPDX-License-Identifier: Apache-2.0<br>
<br>

Strategy Selection
---
*description*

---
The format of the input should look like this:
```python
result_sets = [(anaylsis1results, time1),..., (analysisNresults, timeN)]
```

---

Enter your result sets and the time limit in the next cell as described above.

In [ ]:
result_sets = [('slicing-results/slicing-predicate.2020-11-06_16-54-13.results.reference.ReachSafety-Arrays.xml.bz2', 500), 
               ('slicing-results/slicing-predicate.2020-11-06_16-54-13.results.reference.ReachSafety-Arrays.xml.bz2', 400)]

This cell provides the util methods to convert the result sets to pandas dataframe format.

In [ ]:
from benchexec import tablegenerator
import benchexec.result
import pandas as pd
import numpy as np
from typing import Optional, List, Union


def get_data(data_files: Union[List[str], str]) -> List[pd.DataFrame]:
    if not isinstance(data_files, str):
        return [d for f in data_files for d in get_data(f)]
    raw_data = _load_data(data_files)
    return [pd.DataFrame(data=_get_values(raw_data), columns=_get_column_titles(raw_data))]


def _load_data(data_file: str) -> tablegenerator.RunSetResult:
    parser = tablegenerator.create_argument_parser()
    options = parser.parse_args([data_file])
    return tablegenerator.load_result(data_file, options)


def _get_column_titles(result_set: tablegenerator.RunSetResult):
    return ['task_id', 'category'] + [col.title for col in result_set.columns]


def _get_values(result_set):
    return [[r.task_id[0], r.category] + cast_values(r) for r in result_set.results]


def cast_values(r: tablegenerator.RunResult):
    return [to_value(value, column) for value, column in zip(r.values, r.columns)]


def to_value(value, column):
    if not column.is_numeric() or value is None:
        return value
    return float(value[:-len(column.unit)])


The following methods are used to calculate the final score

In [191]:
def getTestVector():
    cat = ['task_id', 'category', 'status', 'cputime', 'walltime', 'memory', 'host']
    test_vector1 = pd.DataFrame([['task1', 'correct', 'true', 200, 200, 0, 'a'], ['task2', 'error', 'ERROR', 300, 200, 0, 'a']], columns=cat)
    test_vector2 = pd.DataFrame([['task1', 'error', 'ERROR', 200, 200, 0, 'a'], ['task2', 'correct', 'true', 100, 200, 0, 'a']], columns=cat)
    return [(test_vector1, 150), (test_vector2, 400)]

def merge_columns(category, status):
    merge_dict = {
        ("correct", "true"): "TP",
        ("incorrect", "true"): "FP",
        ("correct", "false"): "TN",
        ("incorrect", "true"): "FN"
    }
    if (category, status) in merge_dict:
        return merge_dict[(category, status)]
    return "ERR/UNKNOWN/TIMEOUT"

def prepare_sets(input_sets):
    frames = []
    for (result, time) in input_sets:
        df = get_data(result)[0]
    #Header to use test vector:
    #for (result, time) in getTestVector():
    #    df = result
        df = df.drop(columns=['host'])
        df['status'] = np.where(df['cputime'] > time, 'TIMEOUT', df['status'])
        df['cputime'] = np.where(df['cputime'] > time, time, df['cputime'])
        frames.append(df)
    concat = pd.concat(frames)
    concat['result'] = concat.apply(lambda row: merge_columns(row['category'], row['status']), axis=1)
    return concat

def agg_seq_result(result):
    for elem in result:
        if elem != "ERR/UNKNOWN/TIMEOUT":
            return elem
    return "ERR/UNKNOWN/TIMEOUT"

#fast wrong cputime
def sequential_deprecated():
    results = prepare_sets()
    results = results[['task_id', 'result', 'cputime']]
    # group by id and return first result not equal to "UNKNOWN", sum cputime
    results = results.groupby(results['task_id']).agg({'cputime': 'sum', 'result': lambda x: agg_seq_result(x)})
    return results

def sequential(input_sets):
    prepared = prepare_sets(input_sets)
    prepared = prepared[['task_id', 'result', 'cputime']]
    distinct_ids = prepared.task_id.unique()
    results = pd.DataFrame(columns = prepared.columns)
    # loop through every distinct key
    for key in distinct_ids:
        selection = prepared[prepared['task_id'] == key]
        col_result_distinct = selection.result.unique()
        # if all results are unknown the tasks takes max(cputime)
        if (len(col_result_distinct) == 1):
            if (col_result_distinct[0] == "ERR/UNKNOWN/TIMEOUT"):
                sum_time = selection['cputime'].sum()
                new_row = pd.DataFrame([[key, "ERR/UNKNOWN/TIMEOUT", sum_time]], columns=prepared.columns)
                results = results.append(new_row)
                continue
        # otherwise return result of first analysis that finishes min(cputime | result != "UNKNOWN")
        col_result = ""
        sum_time = 0
        for index, row in selection.iterrows():
            if (row["result"] != "ERR/UNKNOWN/TIMEOUT"):
                col_result = row["result"]
                sum_time = sum_time + row["cputime"]
                break
            sum_time = sum_time + row["cputime"]
        new_row = pd.DataFrame([[key, col_result, sum_time]], columns=prepared.columns)
        results = results.append(new_row)
    return results

def parallel(input_sets):
    prepared = prepare_sets(input_sets)
    prepared = prepared[['task_id', 'result', 'cputime']]
    distinct_ids = prepared.task_id.unique()
    results = pd.DataFrame(columns = prepared.columns)
    # loop through every distinct key
    for key in distinct_ids:
        selection = prepared[prepared['task_id'] == key]
        col_result_distinct = selection.result.unique()
        # if all results are unknown the tasks takes max(cputime)
        if (len(col_result_distinct) == 1):
            if (col_result_distinct[0] == "ERR/UNKNOWN/TIMEOUT"):
                max_time = selection['cputime'].max()
                new_row = pd.DataFrame([[key, "ERR/UNKNOWN/TIMEOUT", max_time]], columns=prepared.columns)
                results = results.append(new_row)
                continue
        # otherwise return result of first analysis that finishes min(cputime | result != "UNKNOWN")
        selection = selection[selection['result'] != "ERR/UNKNOWN/TIMEOUT"]
        selection = selection[selection['cputime'] == selection['cputime'].min()].head(1)
        results = results.append(selection)
    return results

def score(df):
    counts = df.result.value_counts()
    return counts

Now you can choose your prefered strategy.
Initialize
```python
# simulates the sequential execution of all analysis in result_sets
# | analysis 0 | analysis 1 | ... | analysis n |
strategy = sequential()
```
or
```python
# simulates the parallel execution of all analysis in result_sets
# |      analysis 0     |
# |        analysis 1        |
# |       analysis 2       |
# ...
strategy = parallel()
```
to use the sequential or parallel strategy.
Both analysis simulate an execution where the calculation terminates as soon as a result different from "UNKNOWN", "ERROR" or "TIMEOUT" occurs.

In [192]:
strategy = sequential(result_sets)
#strategy = parallel()

Now you can rate your chosen timelimits. The output shows how often a result (e.g. "UNKNOWN, "TP", "FN"...) occured. The score indicates the resulting SV-COMP rating for these results. 

In [193]:
score(strategy)

ERR/UNKNOWN/TIMEOUT    431
TP                       5
Name: result, dtype: int64